## Computing Predictions for Specific Dates and Time Horizons

The plan for replicating Monsaraida’s solution involves creating a customizable Jupyter notebook with input parameters to process the data for training and test datasets and build LightGBM models for predictions. The objective is to train these models to predict values for a specific number of days into the future.

To achieve the best results, the approach is to have each model learn to predict values for a specific week range in the future. As the task requires predicting up to 28 days ahead, the strategy is to create four different models:
- Model 1: Predicts values from day +1 to day +7 in the future.
- Model 2: Predicts values from day +8 to day +14 in the future.
- Model 3: Predicts values from day +15 to +21 in the future.
- Model 4: Predicts values from day +22 to day +28 in the future.

Thus, a separate Kaggle notebook will be used for each of these time ranges, resulting in four notebooks in total. Each notebook will be trained to predict future time spans for each of the 10 stores that were part of the competition, producing ten models per notebook. Consequently, the four notebooks together will generate 40 models, covering all future ranges for all the stores.

Since predictions need to be made for both the public leaderboard and the private one, the entire process is repeated twice:
1. For the public test set submission, training stops at day 1,913, predicting days from 1,914 to 1,941.
2. For the private test set submission, training stops at day 1,941, predicting days from 1,942 to 1,969.

To expedite the process and optimize resource utilization, all eight notebooks can be run in parallel. However, it is crucial to differentiate each notebook based on its name, containing the parameter values relative to the last training day and the look-ahead horizon in days.

The entire workflow, involving data preprocessing, model training, and predicting future values, will be executed across multiple notebooks, allowing for efficient and effective replication of Monsaraida’s solution for accurate time series forecasting on the Kaggle platform.


### import libraries

In [4]:
import numpy as np
import pandas as pd
import os
import random
import math
from decimal import Decimal as dec
import datetime
import time
import gc
import lightgbm as lgb
import pickle

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

### function to reduce the pandas DataFrame memory footprint

In [7]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

Mem. usage decreased to 96.13 Mb (78.8% reduction)
Mem. usage decreased to 143.53 Mb (31.2% reduction)
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to  2.09 Mb (84.5% reduction)


### function helps us to load all the data available and compress it

In [ ]:
def load_data():
    train_df = reduce_mem_usage(pd.read_csv("../data/m5-forecasting-accuracy/sales_train_evaluation.csv"))
    prices_df = reduce_mem_usage(pd.read_csv("../data/m5-forecasting-accuracy/sell_prices.csv"))
    calendar_df = reduce_mem_usage(pd.read_csv("../data/m5-forecasting-accuracy/calendar.csv"))
    submission_df = reduce_mem_usage(pd.read_csv("../data/m5-forecasting-accuracy/sample_submission.csv"))
    return train_df, prices_df, calendar_df, submission_df

In [ ]:
train_df, prices_df, calendar_df, submission_df = load_data()

In [15]:
calendar_df

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [14]:
prices_df

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [11]:
train_df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


### check usage of melt function

In [10]:
index_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
temp_df = pd.melt(train_df, id_vars=index_columns)
temp_df


,id,item_id,dept_id,cat_id,store_id,state_id,variable,value
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0


In [12]:
del(temp_df)
gc.collect()

0

## Function: generate_base_grid(train_df, end_train_day_x, predict_horizon)

This function is designed to generate a base grid for time series data based on the input DataFrame `train_df`, a specific end day `end_train_day_x`, and the number of days to predict into the future `predict_horizon`.

1. **Data Preprocessing:**
   - The `index_columns` list is defined, which contains the column names that will be used as index columns for the grid.
   - The `train_df` DataFrame is transformed using the `pd.melt` function. The data is converted from a wide format to a long format, with the 'd' column representing the day identifier and the 'sales' column representing the sales data.
   - The `reduce_mem_usage` function is called to reduce the memory usage of the DataFrame `grid_df` after the transformation.

2. **Create Holdout Data:**
   - The 'd' column in `grid_df` is temporarily stored in a new column 'd_org' before being modified.
   - The 'd' column in `grid_df` is updated by removing the first two characters from each element and converting the resulting values to 16-bit integers. This is done to convert day identifiers from the format 'd_x' to numerical values.
   - A boolean mask `time_mask` is created based on the 'd' column, which filters rows where the day identifier is within the range from `end_train_day_x` to `end_train_day_x + predict_horizon`.
   - A new DataFrame `holdout_df` is created by selecting specific columns ("id", "d", and "sales") from `grid_df` using the boolean mask `time_mask`. The index of `holdout_df` is reset, and the resulting data is saved as a Feather file named based on the value of `end_train_day_x` and `end_train_day_x + predict_horizon`.

3. **Update Grid Data:**
   - The original `grid_df` is updated by keeping only the rows where the 'd' column is less than or equal to `end_train_day_x`. This step essentially removes data for days after `end_train_day_x`.
   - The 'd' column in `grid_df` is restored to its original values using the 'd_org' column.
   - The 'd_org' column is dropped from `grid_df` as it is no longer needed.

4. **Create Additional Grid Data:**
   - A new DataFrame `add_grid` is created to hold additional data for future days.
   - A loop is executed for each day in the range specified by `predict_horizon`.
   - A temporary DataFrame `temp_df` is created by selecting only the columns in `index_columns` from `train_df` and removing duplicate rows.
   - A new 'd' column is added to `temp_df` with unique day identifiers in the format 'd_x', where 'x' is calculated as `end_train_day_x + i + 1`.
   - A new 'sales' column is added to `temp_df`, and all values are initialized as NaN, indicating that the sales data for these future days is unknown and needs to be predicted or filled later.
   - The temporary DataFrame `temp_df` is concatenated to `add_grid`, accumulating data for each day into the future.

5. **Combine Grid Data:**
   - The original `grid_df` and the newly created `add_grid` are concatenated to create a combined DataFrame, including historical and future data.
   - The index of the combined DataFrame is reset to ensure consecutive integer index values.

6. **Data Type Conversion:**
   - The data type of each column in `grid_df` specified in `index_columns` is converted to the 'category' data type. This can lead to memory optimization and faster operations for categorical data.

7. **Final Data Preparation and Saving:**
   - The memory usage of `grid_df` is reduced using the `reduce_mem_usage` function.
   - The final `grid_df` data is saved as a Feather file named based on the value of `end_train_day_x` and `end_train_day_x + predict_horizon`.

8. **Memory Management:**
   - The function releases the memory occupied by the `holdout_df` and `grid_df` DataFrames using `del` and `gc.collect()` to ensure efficient memory usage.

9. **Print Result:**
   - The function prints the first few rows of the `grid_df` DataFrame using `print(grid_df.head())`.

The `generate_base_grid` function performs data preprocessing, creates a holdout dataset for future predictions, updates the main grid with historical data, adds future data, and saves the final grid data as Feather files for further analysis and modeling in time series forecasting tasks.


In [13]:
def generate_base_grid(train_df, end_train_day_x, predict_horizon):
    index_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

    grid_df = pd.melt(train_df, id_vars=index_columns, var_name='d', value_name='sales')
    grid_df = reduce_mem_usage(grid_df, verbose=False)

    grid_df['d_org'] = grid_df['d']
    # removes the first two characters from each element in the 'd' column
    # first two characters ("d_") have been removed, and the resulting values are converted to 16-bit integers
    grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16)

    time_mask = (grid_df['d'] > end_train_day_x) &  (grid_df['d'] <= end_train_day_x + predict_horizon)
    # # creates a new DataFrame holdout_df by filtering grid_df based on the boolean mask time_mask, keeping only columns "id", "d", and "sales" and resetting the index.
    holdout_df = grid_df.loc[time_mask, ["id", "d", "sales"]].reset_index(drop=True)
    #  lightweight format ideal for fast I/O and interoperability
    holdout_df.to_feather(f"holdout_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
    del(holdout_df)
    gc.collect()

    grid_df = grid_df[grid_df['d'] <= end_train_day_x]
    grid_df['d'] = grid_df['d_org']
    grid_df = grid_df.drop('d_org', axis=1)

    add_grid = pd.DataFrame()
    for i in range(predict_horizon):
        temp_df = train_df[index_columns]
        temp_df = temp_df.drop_duplicates()
        temp_df['d'] = 'd_' + str(end_train_day_x + i + 1)
        temp_df['sales'] = np.nan
        add_grid = pd.concat([add_grid, temp_df])
    
    grid_df = pd.concat([grid_df, add_grid])
    grid_df = grid_df.reset_index(drop=True)
    
    for col in index_columns:
        grid_df[col] = grid_df[col].astype('category')
    
    grid_df = reduce_mem_usage(grid_df, verbose=False)
    grid_df.to_feather(f"grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
    print(grid_df.head())
    del(grid_df)
    gc.collect()

## Function Explanation: merge_by_concat

This function is designed to merge two pandas DataFrames (`df1` and `df2`) based on a common set of columns specified by the `merge_on` parameter. It uses the `pd.merge` function to perform the merge operation and then concatenates the results back to the original DataFrame `df1`. The function returns the merged DataFrame.

### Function Parameters:
- `df1`: The first pandas DataFrame to be merged.
- `df2`: The second pandas DataFrame to be merged with `df1`.
- `merge_on`: A list of column names representing the common columns used for merging the two DataFrames.

### Function Steps:

1. **Extract the Columns to be Merged:**
   - `merged_gf = df1[merge_on]`: Creates a new DataFrame `merged_gf` containing only the columns specified in the `merge_on` list. This DataFrame serves as an intermediate representation of the columns to be merged.

2. **Merge the DataFrames:**
   - `merged_gf = merged_gf.merge(df2, on=merge_on, how='left')`: Performs a left-join merge of `merged_gf` with the DataFrame `df2` based on the columns specified in the `merge_on` list. The `how='left'` parameter ensures that all rows from `merged_gf` are retained, and matching rows from `df2` are appended to it based on the common columns.

3. **Extract New Columns:**
   - `new_columns = [col for col in list(merged_gf) if col not in merge_on]`: Identifies the columns in `merged_gf` that are not part of the `merge_on` list. These columns represent the newly merged data from `df2`.

4. **Concatenate Merged Data with Original DataFrame:**
   - `df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)`: Concatenates the original DataFrame `df1` with the columns in `merged_gf` that are not part of the `merge_on` list. This effectively adds the merged data from `df2` to `df1`.

5. **Return the Merged DataFrame:**
   - `return df1`: The function returns the updated DataFrame `df1`, which now includes the merged data from `df2` based on the specified `merge_on` columns.

### Use Case:
The `merge_by_concat` function can be particularly useful when you want to merge two DataFrames based on specific columns, but you also want to retain all the original data from the first DataFrame (`df1`). By using a left-join merge strategy and concatenating the results back to `df1`, this function ensures that no data from the original DataFrame is lost during the merge process.



In [16]:
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

## Function: calc_release_week(prices_df, end_train_day_x, predict_horizon)

This function calculates the "release week" for items in a given DataFrame and updates the data for a specific range of days. It performs the following steps:

1. **Input Parameters**:
   - `prices_df`: A DataFrame containing information about item prices, including columns like 'store_id', 'item_id', 'wm_yr_wk' (representing the year and week), and other relevant information.
   - `end_train_day_x`: The last day of the training data. The function will calculate the release week up to this day.
   - `predict_horizon`: The number of days into the future to predict. The function will update the data for this future prediction horizon.

2. **Extracting Index Columns**:
   - `index_columns`: A list containing the column names used as index or grouping columns in the DataFrame.

3. **Reading the Data**:
   - The function reads a Feather file named `grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather`, which presumably contains data for a specific range of days.

4. **Calculating the Release Week**:
   - It groups the `prices_df` by 'store_id' and 'item_id' and calculates the minimum 'wm_yr_wk' (year and week identifier) for each item, representing the release week of that item.
   - The result is stored in a new DataFrame called `release_df`, with columns 'store_id', 'item_id', and 'release', representing the release week.

5. **Merging the Release Week Information**:
   - The function merges the `release_df` DataFrame into the original `grid_df` DataFrame, adding the 'release' column to the latter. The merge is done based on the common columns 'store_id' and 'item_id'.

6. **Memory Optimization**:
   - Unnecessary intermediate variables are removed to free up memory.
   - The `reduce_mem_usage` function is applied to optimize memory usage of the DataFrame `grid_df`.

7. **Merging Calendar Information**:
   - The `calendar_df` DataFrame, containing columns 'wm_yr_wk' (year and week identifier) and 'd' (day identifier), is used to merge additional calendar information into the `grid_df` DataFrame based on the common column 'd'.

8. **Calculating Relative Release Week**:
   - The 'release' column in `grid_df` is adjusted to represent the relative release week. It calculates the difference between the 'release' value and the minimum 'release' value and stores the result back in the 'release' column.

9. **Memory Optimization (Again)**:
   - The function optimizes the memory usage of the `grid_df` DataFrame once more using the `reduce_mem_usage` function.

10. **Saving Data**:
    - The updated DataFrame `grid_df` is saved as a Feather file named `grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather`. This file now contains the updated release week information for the given range of days.

11. **Cleaning Up**:
    - Intermediate variables and DataFrames are deleted to free up memory.
    - Garbage collection is invoked to further release memory resources.

Note: Some functions like `merge_by_concat` and `reduce_mem_usage` are used in the function, but their definitions are not provided in this code snippet. These functions likely handle merging DataFrames and optimizing memory usage, respectively.

The purpose of this function is to update the `grid_df` DataFrame with the calculated release week information and other relevant data for a given range of days, making it ready for further analysis or modeling tasks.


In [17]:
def calc_release_week(prices_df, end_train_day_x, predict_horizon):
    index_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
    
    grid_df = pd.read_feather(f"grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
    
    release_df = prices_df.groupby(['store_id', 'item_id'])['wm_yr_wk'].agg(['min']).reset_index()
    release_df.columns = ['store_id', 'item_id', 'release']
    
    grid_df = merge_by_concat(grid_df, release_df, ['store_id', 'item_id'])
    
    del release_df
    grid_df = reduce_mem_usage(grid_df, verbose=False)
    gc.collect()
    
    grid_df = merge_by_concat(grid_df, calendar_df[['wm_yr_wk', 'd']], ['d'])
    grid_df = grid_df.reset_index(drop=True)

    grid_df['release'] = grid_df['release'] - grid_df['release'].min()
    grid_df['release'] = grid_df['release'].astype(np.int16)
    
    grid_df = reduce_mem_usage(grid_df, verbose=False)
    grid_df.to_feather(f"grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
    del(grid_df)
    gc.collect()

### Function: `generate_grid_price(prices_df, calendar_df, end_train_day_x, predict_horizon)`

1. **Input Parameters**:
   - `prices_df`: DataFrame containing historical sell prices for different items in various stores.
   - `calendar_df`: DataFrame containing calendar information, such as week number (`wm_yr_wk`), month, and year.
   - `end_train_day_x`: The last day of the training data.
   - `predict_horizon`: The number of days into the future to predict.

2. **Reading the Existing Grid Data**:
   - The function reads the existing grid DataFrame (`grid_df`) from a Feather file named `"grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather"`.

3. **Calculating Price-Related Features**:
   - The function computes several price-related features for the `prices_df` DataFrame, including 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique', and 'item_nunique'. These features are calculated by grouping data based on store ID and item ID and performing statistical operations on the 'sell_price' column.

4. **Merging Calendar Information with Prices Data**:
   - The function extracts relevant calendar information (`'wm_yr_wk'`, `'month'`, and `'year'`) from `calendar_df` and drops duplicates based on 'wm_yr_wk'.
   - The `prices_df` DataFrame is then merged with the extracted calendar information on 'wm_yr_wk' to add month and year information for each sell price entry.

5. **Calculating Additional Price Features**:
   - The function computes additional price-related features, such as 'price_momentum', 'price_momentum_m', and 'price_momentum_y', which involve measuring the momentum of sell prices based on the previous day, previous month, and previous year, respectively.

6. **Extracting Decimal Part of Prices**:
   - The function extracts the decimal part (cent) of sell prices, 'price_max', and 'price_min' using the `math.modf()` function.

7. **Reducing Memory Usage**:
   - The function optimizes memory usage by reducing the memory footprint of the `prices_df` DataFrame using the `reduce_mem_usage()` function.

8. **Merging Prices Data with Existing Grid**:
   - The function merges the `prices_df` DataFrame with the existing grid (`grid_df`) based on store ID, item ID, and week number ('wm_yr_wk').

9. **Selecting Relevant Columns**:
   - The function selects only the necessary columns ('id', 'd', and other price-related features) from the merged DataFrame.

10. **Reducing Memory Usage for the Final Grid**:
   - The function further optimizes memory usage for the final grid DataFrame by reducing its memory footprint.

11. **Saving the Updated Grid to a Feather File**:
   - The function saves the updated grid DataFrame as a new Feather file named `"grid_price_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather"`.

12. **Memory Management**:
   - The function releases memory occupied by intermediate DataFrames and variables using `del` statements and the `gc.collect()` function.

### Output:
The function performs various data processing and feature engineering steps to create a new grid DataFrame with additional price-related features, and saves the updated grid as a Feather file. The new grid includes data for historical days up to `end_train_day_x` and future days for a range defined by `predict_horizon`, with calculated features based on historical sell prices for different items in various stores.


In [18]:
def generate_grid_price(prices_df, calendar_df, end_train_day_x, predict_horizon):

    grid_df = pd.read_feather(f"grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")

    prices_df['price_max'] = prices_df.groupby(['store_id', 'item_id'])['sell_price'].transform('max')
    prices_df['price_min'] = prices_df.groupby(['store_id', 'item_id'])['sell_price'].transform('min')
    prices_df['price_std'] = prices_df.groupby(['store_id', 'item_id'])['sell_price'].transform('std')
    prices_df['price_mean'] = prices_df.groupby(['store_id', 'item_id'])['sell_price'].transform('mean')
    prices_df['price_norm'] = prices_df['sell_price'] / prices_df['price_max']
    prices_df['price_nunique'] = prices_df.groupby(['store_id', 'item_id'])['sell_price'].transform('nunique')
    prices_df['item_nunique'] = prices_df.groupby(['store_id', 'sell_price'])['item_id'].transform('nunique')

    calendar_prices = calendar_df[['wm_yr_wk', 'month', 'year']]
    calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk'])
    prices_df = prices_df.merge(calendar_prices[['wm_yr_wk', 'month', 'year']], on=['wm_yr_wk'], how='left')
    
    del calendar_prices
    gc.collect()
    
    prices_df['price_momentum'] = prices_df['sell_price'] / prices_df.groupby(['store_id', 'item_id'])[
        'sell_price'].transform(lambda x: x.shift(1))
    prices_df['price_momentum_m'] = prices_df['sell_price'] / prices_df.groupby(['store_id', 'item_id', 'month'])[
        'sell_price'].transform('mean')
    prices_df['price_momentum_y'] = prices_df['sell_price'] / prices_df.groupby(['store_id', 'item_id', 'year'])[
        'sell_price'].transform('mean')

    prices_df['sell_price_cent'] = [math.modf(p)[0] for p in prices_df['sell_price']]
    prices_df['price_max_cent'] = [math.modf(p)[0] for p in prices_df['price_max']]
    prices_df['price_min_cent'] = [math.modf(p)[0] for p in prices_df['price_min']]

    del prices_df['month'], prices_df['year']
    prices_df = reduce_mem_usage(prices_df, verbose=False)
    gc.collect()
    
    original_columns = list(grid_df)
    grid_df = grid_df.merge(prices_df, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
    del(prices_df)
    gc.collect()
    
    keep_columns = [col for col in list(grid_df) if col not in original_columns]
    grid_df = grid_df[['id', 'd'] + keep_columns]
    grid_df = reduce_mem_usage(grid_df, verbose=False)

    grid_df.to_feather(f"grid_price_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
    del(grid_df)
    gc.collect()

### Function: get_moon_phase(d)
This function calculates the moon phase index (0=new moon, 4=full moon) for a given date 'd'.

### Parameters:
- `d`: The input date for which the moon phase needs to be determined. It should be in the format 'YYYY-MM-DD'.

### Dependencies:
- The function uses the `datetime` module to handle date calculations and conversions.
- It also requires the `decimal` module for precise decimal arithmetic and the `math` module for mathematical operations.

### Algorithm:
1. Convert the input date 'd' into the number of days since January 1, 2001, using the difference between the input date and January 1, 2001.
2. Convert the number of days into a decimal value to handle precise arithmetic.
3. Calculate the number of lunations (complete lunar cycles) that have occurred since January 1, 2001, using a formula that takes into account the time elapsed in days.
4. Calculate the phase index by dividing the fractional part of the lunations by 1 and then multiplying it by 8. Round the result to the nearest whole number (0.5 is added before rounding to ensure correct rounding behavior).
5. The phase index ranges from 0 to 7. To ensure the result is within this range, take the bitwise AND operation with 7.
6. Return the final moon phase index as an integer.

### Moon Phases:
- 0: New Moon
- 1: Waxing Crescent
- 2: First Quarter
- 3: Waxing Gibbous
- 4: Full Moon
- 5: Waning Gibbous
- 6: Last Quarter
- 7: Waning Crescent

### Example Usage:
```python
date = "2023-07-31"
moon_phase_index = get_moon_phase(date)
print("Moon Phase Index for", date, ":", moon_phase_index)


In [19]:
def get_moon_phase(d):  # 0=new, 4=full; 4 days/phase
    diff = datetime.datetime.strptime(d, '%Y-%m-%d') - datetime.datetime(2001, 1, 1)
    days = dec(diff.days) + (dec(diff.seconds) / dec(86400))
    lunations = dec("0.20439731") + (days * dec("0.03386319269"))
    phase_index = math.floor((lunations % dec(1) * dec(8)) + dec('0.5'))
    return int(phase_index) & 7
    


### Function Explanation: generate_grid_calendar

This function generates a grid-based calendar DataFrame by merging a pre-existing `calendar_df` with a previously generated `grid_df`, and then performs various data preprocessing steps to prepare the resulting `grid_df` for further analysis or modeling.

#### Arguments:
- `calendar_df`: A DataFrame containing the calendar information, such as dates, events, and snap days (binary indicators of SNAP purchases) for each day.
- `end_train_day_x`: An integer representing the last day of the training data.
- `predict_horizon`: An integer representing the prediction horizon or the number of days to predict into the future.

#### Steps:

1. **Read Existing Grid DataFrame:**
   - The function reads a previously generated Feather file named `grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather` to obtain the existing `grid_df`.

2. **Select Columns:**
   - The function selects only the 'id' and 'd' columns from the existing `grid_df` and assigns the result to `grid_df`.

3. **Memory Management:**
   - The function uses `gc.collect()` to perform garbage collection, freeing up memory.

4. **Calculate Moon Phase:**
   - The function calls the `get_moon_phase` function on the 'date' column of `calendar_df` to determine the moon phase for each date. The result is added to a new column named 'moon' in `calendar_df`.

5. **Merge Calendar Information:**
   - A subset of columns from `calendar_df` is merged with `grid_df` based on the 'd' (day identifier) column, using a left join. The selected columns include 'date', 'd', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', and the newly created 'moon' column.

6. **Data Type Conversion:**
   - Several categorical columns in `grid_df` (e.g., event names, event types, snap indicators) are converted to the 'category' data type to optimize memory usage.

7. **Date Feature Engineering:**
   - The 'date' column in `grid_df` is converted to a datetime data type using `pd.to_datetime`.
   - New temporal features are derived from the 'date' column, including 'tm_d' (day of the month), 'tm_w' (ISO week of the year), 'tm_m' (month), 'tm_y' (year), 'tm_y' (year relative to the minimum year in the dataset), and 'tm_wm' (week of the month).
   - Additionally, 'tm_dw' (day of the week) and 'tm_w_end' (binary indicator for weekend) features are created.

8. **Memory Optimization and Saving:**
   - Unnecessary columns (e.g., 'date') are removed from `grid_df` to save memory.
   - The function calls `reduce_mem_usage` to optimize the memory usage of `grid_df`.
   - The resulting `grid_df` is saved as a new Feather file named `grid_calendar_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather`.

9. **Memory Cleanup:**
   - Unused variables (`grid_df`, `calendar_df`) are deleted to free up memory.
   - Another call to `gc.collect()` performs garbage collection once again.

##### Note:
- The `reduce_mem_usage` function is likely a custom function used to reduce memory usage in the DataFrame by downcasting numerical data types.

- The `get_moon_phase` function is not defined in this code snippet, so it must be defined elsewhere to calculate the moon phase based on the date.

- The function assumes that the `calendar_df` provided has the necessary columns and data for merging and processing. Ensure that the input DataFrame is structured correctly and contains the required information before using this function.

- The resulting `grid_df` will have the additional calendar and date-related features, making it suitable for further analysis, modeling, or time-series forecasting tasks.


In [20]:
def generate_grid_calendar(calendar_df, end_train_day_x, predict_horizon):
    
    grid_df = pd.read_feather(f"grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
    grid_df = grid_df[['id', 'd']]
    gc.collect()

    calendar_df['moon'] = calendar_df.date.apply(get_moon_phase)

    # Merge calendar partly
    icols = ['date',
             'd',
             'event_name_1',
             'event_type_1',
             'event_name_2',
             'event_type_2',
             'snap_CA',
             'snap_TX',
             'snap_WI',
             'moon',
             ]

    grid_df = grid_df.merge(calendar_df[icols], on=['d'], how='left')

    icols = ['event_name_1',
             'event_type_1',
             'event_name_2',
             'event_type_2',
             'snap_CA',
             'snap_TX',
             'snap_WI']
    
    for col in icols:
        grid_df[col] = grid_df[col].astype('category')

    grid_df['date'] = pd.to_datetime(grid_df['date'])

    grid_df['tm_d'] = grid_df['date'].dt.day.astype(np.int8)
    grid_df['tm_w'] = grid_df['date'].dt.isocalendar().week.astype(np.int8)
    grid_df['tm_m'] = grid_df['date'].dt.month.astype(np.int8)
    grid_df['tm_y'] = grid_df['date'].dt.year
    grid_df['tm_y'] = (grid_df['tm_y'] - grid_df['tm_y'].min()).astype(np.int8)
    grid_df['tm_wm'] = grid_df['tm_d'].apply(lambda x: math.ceil(x / 7)).astype(np.int8)

    grid_df['tm_dw'] = grid_df['date'].dt.dayofweek.astype(np.int8)
    grid_df['tm_w_end'] = (grid_df['tm_dw'] >= 5).astype(np.int8)
                                                         
    del(grid_df['date'])
    grid_df = reduce_mem_usage(grid_df, verbose=False)
    grid_df.to_feather(f"grid_calendar_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
                                                         
    del(grid_df)
    del(calendar_df)
    gc.collect()

### Function Explanation: modify_grid_base

This function takes two parameters, `end_train_day_x` and `predict_horizon`, to modify and optimize a DataFrame named `grid_df`. The goal is to reduce memory usage while preserving relevant data for the specified time range.

#### Parameters:
- `end_train_day_x`: The last day of the training data range (an integer).
- `predict_horizon`: The number of days to predict into the future (an integer).

#### Steps:

1. **Reading Data:** The function starts by reading the DataFrame `grid_df` from a Feather file that contains data for the time range from `end_train_day_x` to `end_train_day_x + predict_horizon`. The file is assumed to be in the format "grid_df_end_train_day_x_to_end_train_day_x+predict_horizon.feather".

2. **Day Column Modification:** The 'd' column in `grid_df` contains day identifiers in the format "d_x", where 'x' represents the day number. The function applies a lambda function to each element of the 'd' column to remove the leading "d_" part and converts the remaining day number to a 16-bit integer. This step simplifies the day representation and reduces memory usage.

3. **Removing Unnecessary Column:** The 'wm_yr_wk' column, which likely represents the week number, is not required for the analysis and is deleted from `grid_df` to further save memory.

4. **Memory Optimization:** The function calls the `reduce_mem_usage` function (not shown) to optimize the memory usage of the DataFrame `grid_df`. This step aims to reduce the memory footprint while maintaining data integrity.

5. **Saving Modified Data:** The function saves the modified DataFrame `grid_df` back to a new Feather file with the same naming convention as the input file. This ensures that the optimized data is preserved for future use.

6. **Memory Cleanup:** The function then deletes the DataFrame `grid_df` and performs a garbage collection (`gc.collect()`) to release any unreferenced memory and free up system resources.

#### Return:
The function doesn't have a return value, as it performs in-place modifications on the DataFrame `grid_df`.

Note: The `reduce_mem_usage` function called within this function is not shown in the provided code snippet, but it is presumably defined elsewhere to handle memory optimization.
```

In [21]:
def modify_grid_base(end_train_day_x, predict_horizon):
    grid_df = pd.read_feather(f"grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
    grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16)
    del grid_df['wm_yr_wk']
    
    grid_df = reduce_mem_usage(grid_df, verbose=False)
    grid_df.to_feather(f"grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
    
    del(grid_df)
    gc.collect()

### Function: `generate_lag_feature(end_train_day_x, predict_horizon)`

This function generates lag features and rolling statistics for the sales data in a DataFrame. The generated features are used for time series forecasting and prediction tasks.

**Arguments:**
- `end_train_day_x`: An integer representing the last day of the training data. The function will generate lag features and rolling statistics up to this day.
- `predict_horizon`: An integer representing the number of days into the future for which predictions are to be made.

**Function Steps:**

1. Read Data:
   - The function reads a Feather file named `grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather`, containing the raw sales data for the specified time range. The data is stored in a DataFrame named `grid_df`.
   - The DataFrame `grid_df` is then filtered to retain only the columns "id", "d", and "sales", discarding other irrelevant columns.

2. Generate Lag Features:
   - The function specifies a list of `num_lag_day_list` representing the number of lag days to be used for generating lag features. By default, it takes the 15 days before the `predict_horizon`.
   - For each lag day value in `num_lag_day_list`, it computes the lag feature for the 'sales' column using the `shift()` function on grouped data, and assigns the result to a new column in `grid_df`.

3. Data Type Optimization:
   - The function iterates through the columns of `grid_df`, and if a column name contains 'lag', it converts the data type of that column to `np.float16`. This optimization helps reduce memory usage while preserving data integrity.

4. Generate Rolling Statistics:
   - The function specifies a list of `num_rolling_day_list` representing the window sizes for calculating rolling statistics. It computes the rolling mean and standard deviation for each window size using the `rolling()` function on grouped data, and assigns the results to new columns in `grid_df`.

5. Memory Optimization:
   - The function calls the `reduce_mem_usage()` function to optimize the memory usage of `grid_df`, reducing the memory footprint of the DataFrame.

6. Save Results:
   - The resulting DataFrame `grid_df` with the lag features and rolling statistics is saved to a new Feather file named `lag_feature_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather`.

7. Clean Up:
   - The function releases the memory occupied by `grid_df` using `del` and `gc.collect()` to free up resources.

**Note:**
- The function relies on the existence of a `reduce_mem_usage()` function to optimize DataFrame memory usage, which is not provided in the code snippet. Ensure you have this function defined or replace it with an appropriate memory optimization method.



In [22]:
def generate_lag_feature(end_train_day_x, predict_horizon):
    grid_df = pd.read_feather(f"grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
    grid_df = grid_df[['id', 'd', 'sales']]
    
    num_lag_day_list = []
    num_lag_day = 15
    for col in range(predict_horizon, predict_horizon + num_lag_day):
        num_lag_day_list.append(col)
    
    num_rolling_day_list = [7, 14, 30, 60, 180]
    num_shift_rolling_day_list = []
    for num_shift_day in [1, 7, 14]:
        for num_rolling_day in [7, 14, 30, 60]:
            num_shift_rolling_day_list.append([num_shift_day, num_rolling_day])
   
    grid_df = grid_df.assign(**{
        '{}_lag_{}'.format(col, l): grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(l))
        for l in num_lag_day_list
    })

    for col in list(grid_df):
        if 'lag' in col:
            grid_df[col] = grid_df[col].astype(np.float16)

    for num_rolling_day in num_rolling_day_list:
        grid_df['rolling_mean_' + str(num_rolling_day)] = grid_df.groupby(['id'])['sales'].transform(
            lambda x: x.shift(predict_horizon).rolling(num_rolling_day).mean()).astype(np.float16)
        grid_df['rolling_std_' + str(num_rolling_day)] = grid_df.groupby(['id'])['sales'].transform(
            lambda x: x.shift(predict_horizon).rolling(num_rolling_day).std()).astype(np.float16)

    grid_df = reduce_mem_usage(grid_df, verbose=False)
    grid_df.to_feather(f"lag_feature_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
    
    del(grid_df)
    gc.collect()


### Function: generate_target_encoding_feature(end_train_day_x, predict_horizon)

#### Purpose:
This function generates target encoding features based on historical sales data for a given time period, defined by `end_train_day_x` and `predict_horizon`.

#### Input Parameters:
- `end_train_day_x`: The end day of the training data, represented as an integer (e.g., 100, 200).
- `predict_horizon`: The number of days into the future to predict, represented as an integer (e.g., 7, 14).

#### Function Flow:
1. The function first reads a DataFrame `grid_df` from a Feather file named `grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather`. This file is expected to contain historical sales data for the specified time period.

2. It sets the 'sales' values to NaN for the future days in `grid_df`, i.e., days after `end_train_day_x`, to prepare the target encoding features for future predictions.

3. A list `icols` is defined, containing different combinations of columns to group by for calculating target encoding. These combinations represent different levels of granularity for the target encoding features.

4. For each column combination `col` in `icols`, the function calculates the mean and standard deviation of 'sales' for each group and creates new columns 'enc_{col}_mean' and 'enc_{col}_std' to store the target encoding features.

5. The function reduces the memory usage of the DataFrame `grid_df` to optimize memory consumption.

6. The target encoding features along with the relevant 'id' and 'd' columns are stored in a new Feather file named `target_encoding_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather`.

7. The function performs cleanup by deleting the DataFrame `grid_df` and calling the garbage collector to release unused memory.

#### Notes:
- Target encoding is a technique used in machine learning to transform categorical variables into numerical features based on the target variable (in this case, 'sales'). It helps capture relationships between categorical features and the target variable.
- The function assumes the existence of a custom function `reduce_mem_usage()` that reduces the memory usage of the DataFrame by converting numerical columns to appropriate data types.

#### Example Usage:
```python
# Generate target encoding features for the time period from day 100 to day 107
generate_target_encoding_feature(end_train_day_x=100, predict_horizon=7)


In [23]:
def generate_target_encoding_feature(end_train_day_x, predict_horizon):

    grid_df = pd.read_feather(f"grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
    
    grid_df.loc[grid_df['d'] > (end_train_day_x - predict_horizon), 'sales'] = np.nan
    base_cols = list(grid_df)

    icols = [
        ['state_id'],
        ['store_id'],
        ['cat_id'],
        ['dept_id'],
        ['state_id', 'cat_id'],
        ['state_id', 'dept_id'],
        ['store_id', 'cat_id'],
        ['store_id', 'dept_id'],
        ['item_id'],
        ['item_id', 'state_id'],
        ['item_id', 'store_id']
    ]

    for col in icols:
        col_name = '_' + '_'.join(col) + '_'
        grid_df['enc' + col_name + 'mean'] = grid_df.groupby(col)['sales'].transform('mean').astype(
            np.float16)
        grid_df['enc' + col_name + 'std'] = grid_df.groupby(col)['sales'].transform('std').astype(
            np.float16)

    keep_cols = [col for col in list(grid_df) if col not in base_cols]
    grid_df = grid_df[['id', 'd'] + keep_cols]

    grid_df = reduce_mem_usage(grid_df, verbose=False)
    grid_df.to_feather(f"target_encoding_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
    
    del(grid_df)
    gc.collect()

# Assemble Grid by Store Function

The `assemble_grid_by_store` function is designed to assemble a new grid data for each store based on the input data from the `train_df` DataFrame. It combines multiple feather files containing different components of the grid data and performs some data manipulation and merging operations.

### Parameters:

- `train_df`: The input DataFrame containing the training data for different stores.
- `end_train_day_x`: An integer representing the end day of the training data.
- `predict_horizon`: An integer indicating the number of days into the future to predict.

### Function Overview:

1. **Concatenating DataFrames:**
   The function starts by concatenating three feather files using `pd.concat`. These files contain different components of the grid data:
   - A feather file containing the main grid data (`grid_df`).
   - A feather file containing price data for products (`grid_price`).
   - A feather file containing calendar data (`grid_calendar`).
   The resulting DataFrame is stored in `grid_df`.

2. **Creating a Store ID List:**
   The function creates a list of unique store IDs from the 'store_id' column in `train_df`. This list will be used to process data for each store separately.

3. **Extracting Data for Each Store:**
   For each store ID in the list, the function extracts the corresponding data from `grid_df` and stores it in a dictionary called `index_store`. This dictionary stores the extracted data along with their original indices, which will be used later for merging data.

4. **Saving Grid Data for Each Store:**
   The function then iterates through the store ID list and processes data for each store individually. It loads the extracted grid data for the specific store from the dictionary, resets the index, and saves the data to a new feather file with a store-specific filename.

5. **Target Encoding Features:**
   The function loads the target encoding features (mean and standard deviation) from a separate feather file (`target_encoding`) and stores them in a DataFrame `df2`.

6. **Merging Target Encoding Features:**
   For each store ID, the function loads the corresponding grid data, merges it with the target encoding features for that store (using the pre-saved indices from `index_store`), and saves the updated data to a new feather file with a store-specific filename.

7. **Lag Features:**
   The function loads lag features data from a separate feather file (`lag_feature`) and stores them in a DataFrame `df3`.

8. **Merging Lag Features:**
   Similar to the previous step, the function loads the grid data for each store, merges it with the lag features for that store (using the pre-saved indices from `index_store`), and saves the updated data to a new feather file with a store-specific filename.

### Memory Management:

Throughout the function, the script uses `gc.collect()` to release memory used by intermediate data and DataFrames after they are no longer needed. This helps manage memory usage efficiently when dealing with large datasets.

Overall, the function helps create separate grid data for each store by combining various components, merging target encoding and lag features, and saves the final grid data for each store into separate feather files for later use in modeling or analysis.

Please note that the function assumes the existence of specific feather files with the appropriate filenames, which were likely created in earlier steps of the data preparation process.


In [24]:
def assemble_grid_by_store(train_df, end_train_day_x, predict_horizon):
    grid_df = pd.concat([pd.read_feather(f"grid_df_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather"),
                     pd.read_feather(f"grid_price_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather").iloc[:, 2:],
                     pd.read_feather(f"grid_calendar_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather").iloc[:, 2:]],
                     axis=1)
    gc.collect()
    store_id_set_list = list(train_df['store_id'].unique())

    index_store = dict()
    for store_id in store_id_set_list:
        extract = grid_df[grid_df['store_id'] == store_id]
        index_store[store_id] = extract.index.to_numpy()
        extract = extract.reset_index(drop=True)
        extract.to_feather(f"grid_full_store_{store_id}_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")

    del(grid_df)
    gc.collect()
    
    mean_features = [
        'enc_cat_id_mean', 'enc_cat_id_std',
        'enc_dept_id_mean', 'enc_dept_id_std',
        'enc_item_id_mean', 'enc_item_id_std'
        ]
    df2 = pd.read_feather(f"target_encoding_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")[mean_features]

    for store_id in store_id_set_list:
        df = pd.read_feather(f"grid_full_store_{store_id}_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
        df = pd.concat([df, df2[df2.index.isin(index_store[store_id])].reset_index(drop=True)], axis=1)
        df.to_feather(f"grid_full_store_{store_id}_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")

    del(df2)
    gc.collect()
    
    df3 = pd.read_feather(f"lag_feature_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather").iloc[:, 3:]

    for store_id in store_id_set_list:
        df = pd.read_feather(f"grid_full_store_{store_id}_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
        df = pd.concat([df, df3[df3.index.isin(index_store[store_id])].reset_index(drop=True)], axis=1)
        df.to_feather(f"grid_full_store_{store_id}_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")

    del(df3)
    del(store_id_set_list)
    gc.collect()


### Function Explanation: load_grid_by_store

This function loads data from a Feather file representing sales data for a specific store within a given time range. It performs some preprocessing and memory optimization before returning the final DataFrame and a list of enabled features.

#### Parameters:
- `end_train_day_x`: The last day of the training data for the store. (int)
- `predict_horizon`: The number of days to predict into the future. (int)
- `store_id`: The unique identifier of the store for which data is loaded. (str)

#### Function Steps:
1. **Read Data from Feather File:**
   - The function reads the sales data for the specified store and time range from a Feather file named `grid_full_store_{store_id}_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather`. This file should contain columns like 'id', 'd', 'sales', and additional features.

2. **Remove Unnecessary Features:**
   - Some features like 'id', 'state_id', 'store_id', 'date', 'wm_yr_wk', 'd', and 'sales' are not needed for further processing or modeling. These features are stored in the `remove_features` list.
   - The function creates a list of `enable_features` containing all the features from the DataFrame except the ones in the `remove_features` list.

3. **Rearrange Columns and Reduce Memory Usage:**
   - The DataFrame `df` is rearranged to have columns in the following order: 'id', 'd', 'sales', and then the remaining `enable_features`. This ensures that the target variable 'sales' and essential information about each row are placed at the beginning of the DataFrame.
   - The function applies the `reduce_mem_usage` function to optimize memory usage in the DataFrame. This function reduces the memory footprint of numerical columns by using appropriate data types without compromising data integrity. It is likely a custom memory optimization function implemented elsewhere in the code.

4. **Garbage Collection and Return:**
   - The function performs garbage collection (`gc.collect()`) to free up any memory that is no longer in use after the DataFrame has been processed.
   - Finally, the function returns the processed DataFrame `df` and the list of `enable_features`, which will be used in later stages of the data analysis or modeling process.

#### Example Usage:
```python
store_data, features = load_grid_by_store(1913, 28, "CA_1")


In [25]:
def load_grid_by_store(end_train_day_x, predict_horizon, store_id):
    df = pd.read_feather(f"grid_full_store_{store_id}_{end_train_day_x}_to_{end_train_day_x + predict_horizon}.feather")
                          
    remove_features = ['id', 'state_id', 'store_id', 'date', 'wm_yr_wk', 'd', 'sales']
    enable_features = [col for col in list(df) if col not in remove_features]
    df = df[['id', 'd', 'sales'] + enable_features]
    df = reduce_mem_usage(df, verbose=False)
    gc.collect()
                          
    return df, enable_features

### Function: train(train_df, seed, end_train_day_x, predict_horizon)

This function is used for training LightGBM models to predict sales for different store locations. It takes the following parameters:
- `train_df`: The input DataFrame containing the training data.
- `seed`: The random seed used for reproducibility in model training.
- `end_train_day_x`: The last day of the training data, denoted as "d_x", where "x" is a day identifier.
- `predict_horizon`: The prediction horizon, representing the number of days into the future to make predictions for.

#### LightGBM Parameters:

The function defines a dictionary `lgb_params` containing various hyperparameters for the LightGBM model. These parameters include the boosting type, objective function, learning rate, number of leaves, feature fraction, and more. The model is configured to use the "tweedie" objective with a variance power of 1.1 and "goss" boosting type for gradient-based one-side sampling.

#### Random Seed and Environment Settings:

The random seed and environment settings are initialized to ensure reproducibility in the training process across runs. The random seed is set for Python, NumPy, and LightGBM.

#### Training Process:

The function loops over each unique store ID in the input DataFrame `train_df`. For each store, it proceeds as follows:

1. **Data Loading and Preparation:**
   - The function calls `load_grid_by_store()` to load the data for the specific store based on `end_train_day_x` and `predict_horizon`. It also retrieves a list of enable features (columns) for the training.

2. **Data Splitting:**
   - The data is split into three parts: training, validation, and prediction. Training data includes rows where the day identifier is less than or equal to `end_train_day_x`. Validation data includes rows where the day identifier is within the last `predict_horizon` days of the training data. Prediction data is used for later predictions and includes rows with day identifiers beyond the last 100 days of the training data.

3. **Model Training:**
   - A LightGBM model is trained using the training data and validated using the validation data. The trained model is saved to a binary file named `lgb_model_{store_id}_{predict_horizon}.bin`.

4. **Feature Importance:**
   - The function calculates feature importance scores based on the trained model and stores them in a CSV file named `feature_importance_{store_id}_{predict_horizon}.csv`. These scores show the relative importance of each feature in predicting sales for the specific store.

5. **Aggregating Feature Importance:**
   - The feature importance scores for all stores are aggregated and saved in a CSV file named `feature_importance_all_{predict_horizon}.csv`. Additionally, a summary of mean and standard deviation of feature importance scores across all stores is saved to `feature_importance_agg_{predict_horizon}.csv`.

The training process is repeated for each unique store, and at the end of the function execution, the trained models and feature importance information are available for each store, as well as their aggregation.

The function's output includes trained models, feature importance files, and aggregated feature importance files, which can be used for further analysis, model evaluation, or making predictions on the test dataset.


For training time reasons, we just modified the boosting type, choosing to use Gradient-Based One-Side Sampling (GOSS) instead of Gradient Boosting Decision Tree (GBDT) because that can really speed up training without much loss in terms of performance. A good speed-up to the
model is also provided by the subsample parameter and the feature fraction: at each learning step of the gradient boosting, only half of the examples and half of the features will be considered. 
The Tweedie loss, with a power value of 1.1 (hence with an underlying distribution closer to Poisson) seems particularly effective in modeling intermittent series (where zero sales prevail). The used metric is just the root mean squared error (there is no necessity to use a custom metric for representing the competition metric). We also use the force_row_wise parameter to save memory in the Kaggle notebook. All the other parameters are exactly the ones presented by Monsaraida in his solution (apart from the subsampling parameter that has been disabled because of its
incompatibility with the goss boosting type).

In [30]:
def train(train_df, seed, end_train_day_x, predict_horizon):
    
    lgb_params = {
            'boosting_type': 'goss',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            #'subsample': 0.5, # incompatibility with the goss boosting type
            #'subsample_freq': 1, # incompatibility with the goss boosting type
            'learning_rate': 0.03,
            'num_leaves': 2 ** 11 - 1,
            'min_data_in_leaf': 2 ** 12 - 1,
            'feature_fraction': 0.5,
            'max_bin': 100,
            'boost_from_average': False,
            'num_boost_round': 1400,
            'verbose': -1,
            'num_threads': os.cpu_count(),
            'force_row_wise': True,
        }

    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    lgb_params['seed'] = seed

    store_id_set_list = list(train_df['store_id'].unique())
    print(f"training stores: {store_id_set_list}")
    
    feature_importance_all_df = pd.DataFrame()
    for store_index, store_id in enumerate(store_id_set_list):
        print(f'now training {store_id} store')

        grid_df, enable_features = load_grid_by_store(end_train_day_x, predict_horizon, store_id)

        train_mask = grid_df['d'] <= end_train_day_x
        valid_mask = train_mask & (grid_df['d'] > (end_train_day_x - predict_horizon))
        preds_mask = grid_df['d'] > (end_train_day_x - 100)

        train_data = lgb.Dataset(grid_df[train_mask][enable_features],
                                 label=grid_df[train_mask]['sales'])

        valid_data = lgb.Dataset(grid_df[valid_mask][enable_features],
                                 label=grid_df[valid_mask]['sales'])


        # Saving part of the dataset for later predictions
        # Removing features that we need to calculate recursively
        grid_df = grid_df[preds_mask].reset_index(drop=True)
        grid_df.to_feather(f'test_{store_id}_{predict_horizon}.feather')
        del(grid_df)
        gc.collect()
        
        estimator = lgb.train(lgb_params,
                              train_data,
                              valid_sets=[valid_data],
                              callbacks=[lgb.log_evaluation(period=100, show_stdv=False)],
                              )

        model_name = str(f'lgb_model_{store_id}_{predict_horizon}.bin')
        feature_importance_store_df = pd.DataFrame(sorted(zip(enable_features, estimator.feature_importance())),
                                                   columns=['feature_name', 'importance'])
        feature_importance_store_df = feature_importance_store_df.sort_values('importance', ascending=False)
        feature_importance_store_df['store_id'] = store_id
        feature_importance_store_df.to_csv(f'feature_importance_{store_id}_{predict_horizon}.csv', index=False)
        feature_importance_all_df = pd.concat([feature_importance_all_df, feature_importance_store_df])
        pickle.dump(estimator, open(model_name, 'wb'))

        del([train_data, valid_data, estimator])
        gc.collect()

    feature_importance_all_df.to_csv(f'feature_importance_all_{predict_horizon}.csv', index=False)
    feature_importance_agg_df = feature_importance_all_df.groupby('feature_name')['importance'].agg(['mean', 'std']).reset_index()
    feature_importance_agg_df.columns = ['feature_name', 'importance_mean', 'importance_std']
    feature_importance_agg_df = feature_importance_agg_df.sort_values('importance_mean', ascending=False)
    feature_importance_agg_df.to_csv(f'feature_importance_agg_{predict_horizon}.csv', index=False)

### Function: train_pipeline(train_df, prices_df, calendar_df, end_train_day_x_list, prediction_horizon_list)

This function is the main pipeline that executes a series of data preparation and modeling steps for training a forecasting model. It takes the following parameters:
- `train_df`: The DataFrame containing historical sales data, which is used as the training data for the model.
- `prices_df`: The DataFrame containing historical price information for the products.
- `calendar_df`: The DataFrame containing calendar information, such as dates and events.
- `end_train_day_x_list`: A list of end points for the training data, each representing a specific day.
- `prediction_horizon_list`: A list of prediction horizons, each representing the number of days to predict into the future.

The function performs the following steps for each combination of `end_train_day_x` and `predict_horizon`:

1. **Data Preparation:**
   - `generate_base_grid(train_df, end_train_day_x, predict_horizon)`: Generates a base grid containing all possible combinations of products, stores, and days up to `end_train_day_x` and extends it to `predict_horizon` days into the future.
   - `calc_release_week(prices_df, end_train_day_x, predict_horizon)`: Calculates the release week for each product, which represents the week when the product was introduced to the market, based on historical price information.
   - `generate_grid_price(prices_df, calendar_df, end_train_day_x, predict_horizon)`: Generates additional features related to prices by merging historical price data with calendar information for the given time range.
   - `generate_grid_calendar(calendar_df, end_train_day_x, predict_horizon)`: Generates calendar-related features by merging calendar information with the grid data for the given time range.
   - `modify_grid_base(end_train_day_x, predict_horizon)`: Modifies the base grid by adding lag features and other relevant data to enhance the feature set.
   - `generate_lag_feature(end_train_day_x, predict_horizon)`: Generates lag features by incorporating historical sales data for a specified number of days before the target day.
   - `generate_target_encoding_feature(end_train_day_x, predict_horizon)`: Generates target encoding features, which involve using aggregated sales information for different categorical variables.
   - `assemble_grid_by_store(train_df, end_train_day_x, predict_horizon)`: Assembles the final grid by combining the modified base grid with the historical sales data up to `end_train_day_x`.

2. **Modelling:**
   - `train(train_df, seed, end_train_day_x, predict_horizon)`: Calls a function to train a forecasting model using the assembled grid data, a random seed value (`seed`), and the specified `end_train_day_x` and `predict_horizon`.

Note: The output of each step may modify or extend the original `train_df` DataFrame, allowing for a comprehensive feature set to be created before training the model.

The function provides a convenient way to iterate through different combinations of end points and prediction horizons and carry out a series of data preparation steps, followed by model training for each combination.


In [29]:
def train_pipeline(train_df, prices_df, calendar_df, end_train_day_x_list, prediction_horizon_list, seed):
    
    for end_train_day_x in end_train_day_x_list:
        
        for predict_horizon in prediction_horizon_list:
            
            print(f"end training point day: {end_train_day_x} - prediction horizon: {predict_horizon} days")

            # Data preparation
            generate_base_grid(train_df, end_train_day_x, predict_horizon)
            calc_release_week(prices_df, end_train_day_x, predict_horizon)
            generate_grid_price(prices_df, calendar_df, end_train_day_x, predict_horizon)
            generate_grid_calendar(calendar_df, end_train_day_x, predict_horizon)
            modify_grid_base(end_train_day_x, predict_horizon)
            generate_lag_feature(end_train_day_x, predict_horizon)
            generate_target_encoding_feature(end_train_day_x, predict_horizon)
            assemble_grid_by_store(train_df, end_train_day_x, predict_horizon)

            # Modelling
            train(train_df, seed, end_train_day_x, predict_horizon)
    

In [32]:
end_train_day_x_list = [1913] # [1941, 1913, 1885, 1857, 1829, 1577]
prediction_horizon_list = [7] # [7, 14, 21, 28]
seed = 42

train_pipeline(train_df, prices_df, calendar_df, end_train_day_x_list, prediction_horizon_list, seed)

end training point day: 1913 - prediction horizon: 7 days
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id    d  sales  
0       CA  d_1    0.0  
1       CA  d_1    0.0  
2       CA  d_1    0.0  
3       CA  d_1    0.0  
4       CA  d_1    0.0  
training stores: ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
now training CA_1 store
[100]	valid_0's rmse: 1.922
[200]	valid_0's rmse: 1.91129
[300]	valid_0's rmse: 1.90334
[400]	valid_0's rmse: 1.89908
[500]	valid_0's rmse: 1.89361
[600]	valid_0's rmse: 1.88909
[700]	valid

After defining the training parameters, we just iterate over the stores, each time uploading the
training data of a single store and training the LightGBM model. Each model is then pickle dumped
(saved). We also extract feature importance from each model in order to consolidate it into a
file and then aggregate it, resulting in having the mean importance across all the stores for that
prediction horizon for each feature.